# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,gazi,35.33,25.07,26.11,78,40,2.60,GR,1601355047
1,bluff,-46.60,168.33,5.56,73,99,9.39,NZ,1601355047
2,cape town,-36.41,-71.90,6.00,100,100,1.00,CL,1601355048
3,porto velho,41.15,-8.61,14.44,93,48,1.00,PT,1601355049
4,mataura,-46.19,168.86,5.56,73,94,9.39,NZ,1601355049


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,35.33,25.07
1,-46.60,168.33
2,-36.41,-71.90
3,41.15,-8.61
4,-46.19,168.86


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0     78
1     73
2    100
3     93
4     73
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 17


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,geraldton,-28.77,114.60,22.00,53,0,5.10,AU,1601355083
68,benghazi,32.12,20.07,25.18,81,0,6.97,LY,1601355099
81,gilgit,35.92,74.31,21.79,22,0,1.89,PK,1601355108
98,jiwani,25.05,61.74,28.73,67,0,2.38,PK,1601355119
164,tiznit,29.58,-9.50,21.15,45,0,0.97,MA,1601355165
193,mocuba,-16.84,36.99,26.88,45,0,1.39,MZ,1601355188
216,damnagar,21.70,71.52,28.86,64,0,3.09,IN,1601355204
230,caico,-6.46,-37.10,22.40,77,0,5.61,BR,1601355214
236,darnah,32.77,22.64,25.00,82,0,6.30,LY,1601355218
254,saint-louis,16.33,-15.00,27.23,68,0,3.67,SN,1601355231


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
48,geraldton,-28.77,114.60,AU,
68,benghazi,32.12,20.07,LY,
81,gilgit,35.92,74.31,PK,
98,jiwani,25.05,61.74,PK,
164,tiznit,29.58,-9.50,MA,
193,mocuba,-16.84,36.99,MZ,
216,damnagar,21.70,71.52,IN,
230,caico,-6.46,-37.10,BR,
236,darnah,32.77,22.64,LY,
254,saint-louis,16.33,-15.00,SN,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for geraldton
--------------------------
Finding nearest hotel for benghazi
--------------------------
Finding nearest hotel for gilgit
--------------------------
Finding nearest hotel for jiwani
--------------------------
Finding nearest hotel for tiznit
--------------------------
There are no Hotels for tiznit, moving on to the next city....

Finding nearest hotel for mocuba
--------------------------
Finding nearest hotel for damnagar
--------------------------
Finding nearest hotel for caico
--------------------------
Finding nearest hotel for darnah
--------------------------
Finding nearest hotel for saint-louis
--------------------------
There are no Hotels for saint-louis, moving on to the next city....

Finding nearest hotel for atar
--------------------------
Finding nearest hotel for turayf
--------------------------
Finding nearest hotel for pokhara
--------------------------
Finding nearest hotel for turbat
------------------

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
48,geraldton,-28.77,114.60,AU,The Gerald Apartment Hotel
68,benghazi,32.12,20.07,LY,Tibesti Hotel
81,gilgit,35.92,74.31,PK,Gilgit Serena Hotel
98,jiwani,25.05,61.74,PK,Victoria Hut
193,mocuba,-16.84,36.99,MZ,Pensão Venus
216,damnagar,21.70,71.52,IN,Gujarat tea centre
230,caico,-6.46,-37.10,BR,Hotel Porto Bello 2
236,darnah,32.77,22.64,LY,فندق النعيم
367,atar,20.52,-13.05,MR,Odar kanawal
405,turayf,31.67,38.66,SA,Golden Dune Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))